In [1]:
import sagemaker
from sagemaker.tensorflow import TensorFlow
from sagemaker.tuner import HyperparameterTuner, IntegerParameter, ContinuousParameter, CategoricalParameter
import boto3

# Define the SageMaker session and role
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

# S3 bucket for saving model artifacts and output
bucket = "kedrobucket"
instance_type = "ml.g4dn.4xlarge"

input_path = "s3://kedrobucket/supply_chain_data_asset_shr/training/jane_st_forecasting"
output_path = "s3://kedrobucket/supply_chain_data_asset_shr/training/jane_st_forecasting"

# Define the TensorFlow estimator
estimator = TensorFlow(
    entry_point='autoencoder_tuner.py',  # Path to your Python script
    source_dir="tuner",
    role=role,
    framework_version='2.16',
    py_version='py310',
    instance_count=1,
    instance_type=instance_type,
    output_path=output_path,
    script_mode=True,
    model_dir=False,
    hyperparameters={
        'epochs': 1,  # Default values for static parameters
        'batch_size': 32,
        "s3_filename": "df_530_lite.npz",
        "learning_rate": 1e-3
    }
)

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/base_serializers.py:28: UserWarning: A NumPy version >=1.23.5 and <2.3.0 is required for this version of SciPy (detected version 1.22.4)
  import scipy.sparse


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [2]:
# Define hyperparameter ranges for tuning
hyperparameter_ranges = {
    # 'learning_rate': ContinuousParameter(0.0001, 0.01),
    'latent_dim': CategoricalParameter([8, 10]),
    # 'n_enc_1': IntegerParameter(32, 128),
    # 'n_enc_2': IntegerParameter(16, 64),
    # 'n_dec_1': IntegerParameter(16, 64),
    # 'n_dec_2': IntegerParameter(32, 128)
}

# Define the metric that SageMaker will optimize (mean_cv_error)
objective_metric_name = 'mean_cv_error'
objective_type = 'Minimize'
metric_definitions = [{'Name': 'mean_cv_error', 'Regex': 'mean_cv_error=([0-9\\.]+)'}]

# Create a hyperparameter tuner
tuner = HyperparameterTuner(
    estimator,
    objective_metric_name=objective_metric_name,
    hyperparameter_ranges=hyperparameter_ranges,
    max_jobs=2,  # Number of tuning jobs
    max_parallel_jobs=1,  # Number of parallel jobs
    objective_type=objective_type,
    metric_definitions=metric_definitions
)

In [3]:
# Launch the tuning job
tuner.fit({"training": input_path}, wait=True)

No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config
No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config


ClientError: An error occurred (ValidationException) when calling the CreateHyperParameterTuningJob operation: You don't have enough combinations of hyperparameter ranges. The total number of hyperparameter combinations for the provided ranges [2.0] must be equal or greater than the value of MaxNumberOfJobs, [20]. Provide additional ranges.

In [ ]:
import boto3

sagemaker = boto3.client('sagemaker')

response = sagemaker.list_training_jobs_for_hyper_parameter_tuning_job(
    HyperParameterTuningJobName='tensorflow-training-241023-2208'
)
training_jobs = response['TrainingJobSummaries']
print(training_jobs)

In [ ]:
# !aws sagemaker stop-hyper-parameter-tuning-job --hyper-parameter-tuning-job-name {}

In [ ]:
training_jobs[0]

In [ ]:
response = sagemaker.describe_training_job(TrainingJobName='tensorflow-training-241023-2208-005-cebae95b')
hyperparameters = response['HyperParameters']
print(hyperparameters)